# Using Synthea

Synthea is a synthetic data set that models the population of the state of [Massachusetts](https://en.wikipedia.org/wiki/Massachusetts) in the USA. The [source code](https://github.com/synthetichealth/synthea) used to generate this data is open source and can be adopted to other populations.

The data we are using is a small subset from the overall Synthea data set.

In this notebook we will use [Pandas](https://pandas.pydata.org/) to read in and visualize some of the data. We are going to doing a very simple approach; much better analyses could be conducted.


In [ ]:
import pandas as pd
import itables
from venn import venn
import ipywidgets as ipw
import os
from IPython.display import display, clear_output, HTML
%matplotlib inline

def report_alert_frac(alerts, reference):
    return "%3.1f%%"%(100*(len(alerts)/ len(reference)))
def view_case(enc, data):
    #clear_output()
    rslt = ""
    for k,v in data.items():
        rslt = rslt + "<h3>%s</h3>\n"%k
        rslt = rslt + v[v["ENCOUNTER"]==enc].to_html()
        rslt = rslt +"<hr>\n"
    display(HTML(rslt))

#### Read in the Synthea data

In [ ]:
dfs = ["patient_encounters.csv", 'medications.csv',  'observations.csv', 'conditions.csv', 'procedures.csv']
encdata = {os.path.splitext(f)[0]:pd.read_csv(f) for f in dfs}


## Examing tEMR activity

In a separate activity, we designed an alert to notify physicians in outpatient clinics that they should consider ordering an [A1C test](https://en.wikipedia.org/wiki/Glycated_hemoglobin#Measurement) for the current patient as a screening for diabetes.

Our alert was limited to 

- AGE > 45
- BMI > 25

## What kind of encounters do we have?

In [ ]:
encdata["patient_encounters"].drop_duplicates(subset='ENCOUNTER', keep='first')["ENCOUNTERCLASS"].value_counts().plot.bar()

### Before going further let us limit data to outpatient encounters


In [ ]:
encdata["patient_encounters"] = encdata["patient_encounters"][encdata["patient_encounters"]["ENCOUNTERCLASS"].isin(["wellness", "ambulatory", "outpatient"])]
encdata["patient_encounters"].drop_duplicates(subset='ENCOUNTER', keep='first')["ENCOUNTERCLASS"].value_counts().plot.bar()

### Drop all the other data that doesn't correspond to outpatient encounters

In [ ]:
outpatientids = set(encdata["patient_encounters"]["ENCOUNTER"])
for k in encdata.keys():
    if k != "patient_encounters":
        df = encdata[k]
        encdata[k] = df[df["ENCOUNTER"].isin(outpatientids)]

### What is the distribution of ages in the data set?

In [ ]:
encdata["patient_encounters"]["AGE@ENC"].plot.hist(bins=50)

### What is the distribution of BMI in the data set?

In [ ]:
df = encdata["observations"]
df[df["DESCRIPTION"]=="Body Mass Index"]["VALUENUMERIC"].plot.hist(bins=50)

### Other features

In [ ]:
encdata["patient_encounters"]["RACE"].value_counts().plot.bar()

In [ ]:
 encdata["patient_encounters"]["GENDER"].value_counts().plot.bar()

### Use itables to explore the data

- `nan` indicates a missing value
- The data set is too large to explore in its entirety, so I'm randomly sampling 200 rows; repeat running the cell as many times as you like or change n (but not too large). 

In [ ]:
@ipw.interact(k=encdata.keys(), n=(20, 1000, 20))
def view_tables(k, n):
    itables.show(encdata[k].sample(n=n), maxColumns=0)

### How many encounters and patients do we have?

In [ ]:
len(encdata["patient_encounters"]['ENCOUNTER'].unique())

In [ ]:
len(encdata["patient_encounters"]['PATIENT'].unique())

# Simple Exploration of Cohorts and Alert Frequency

The following portion of the notebook explores a (relatively) simple way to explore the cohort we would identify/alerts we would generate. This exploration ignores all temporal information (e.g., we cannot look back at at values/observations/procedures at a previous encounter).

We will use [sets](https://en.wikipedia.org/wiki/Set_(mathematics)) to create unique collections of encounter IDs and then use set operations to combine these. Sets are useful because they do not contain duplicate values. Thus $\left\{ 1, 1, 2, 3, 3 \right\} = \left\{1, 2, 3 \right\}$

- Union ($A \cup B$): The set of all elements that are in $A$ OR $B$.
- Intersecton ($A \cap B$: The set of all elements are are in $A$ AND $B$
- Difference ($A \setminus B$): The set of all element that are in $A$ and are NOT in $B$

#### Examples



In [ ]:
A = {"Brian", "Wendy", "Susan", "Daniel"}
B = {"Brian", "Marta", "Matt", "Dennis", "Chris"}
C = {"Daniel", "Javeria", "Kathleen"}

print("%s %s"%("A OR B:".ljust(20), str(A.union(B)).rjust(75)))

print("%s %s"%("A AND B:".ljust(20), str(A.intersection(B)).rjust(75)))

print("%s %s"%("(A OR B) and NOT C:".ljust(20), str((A.union(B)).difference(C)).rjust(75)))

print("%s %s"%("(A AND B) and NOT C:".ljust(20), str((A.intersection(B)).difference(C)).rjust(75)))

print("%s %s"%("(A AND B) OR C:".ljust(20), str((A.intersection(B)).union(C)).rjust(75)))

## What are other data we could filter on?

### What are our conditions?

In [ ]:
cons = list(encdata["conditions"]["DESCRIPTION"].dropna().unique())
cons.sort()
for c in cons:
    print(c)

### Diabetes conditions

In [ ]:
for d in encdata["conditions"]["DESCRIPTION"].dropna().unique():
    if 'diabetes' in d.lower():
        print(d)

### What are our observations?

In [ ]:
obs = list(encdata["observations"]["DESCRIPTION"].dropna().unique())
obs.sort()
for o in obs:
    print(o)

#### Potentially useful

- Body mass index (BMI) [Percentile] Per age and gender
- Glucose
- Hemoglobin A1c/Hemoglobin.total in Blood

### What are our medications?

In [ ]:
meds = list(encdata["medications"]["DESCRIPTION"].dropna().unique())
meds.sort()
for m in meds:
    print(m)

In [ ]:
for m in encdata["medications"]["DESCRIPTION"].dropna().unique():
    if 'insul' in m.lower():
        print(m)

### What are our procedures?

In [ ]:
pros = list(encdata["procedures"]["DESCRIPTION"].dropna().unique())
pros.sort()
for p in pros:
    print(p)

In [ ]:
for p in encdata["procedures"]["DESCRIPTION"].dropna().unique():
    if 'diabetes' in p.lower():
        print(p)

## Create a set with all encounters

In [ ]:
all_enc = set(encdata["patient_encounters"]["ENCOUNTER"])

## Potential exclusions

In [ ]:
diabetes_conds= [
"Diabetes",
"Neuropathy due to type 2 diabetes mellitus (disorder)",
"Diabetic retinopathy associated with type II diabetes mellitus (disorder)",
"Nonproliferative diabetic retinopathy due to type 2 diabetes mellitus (disorder)",
"Microalbuminuria due to type 2 diabetes mellitus (disorder)",
"Macular edema and retinopathy due to type 2 diabetes mellitus (disorder)",
"Proliferative diabetic retinopathy due to type II diabetes mellitus (disorder)"
]

insulin = [
"insulin human  isophane 70 UNT/ML / Regular Insulin  Human 30 UNT/ML Injectable Suspension [Humulin]",
"Insulin Lispro 100 UNT/ML Injectable Solution [Humalog]"
]

screen = ["Urine screening test for diabetes"]

In [ ]:
df = encdata["conditions"]
diabetes = set(df[df["DESCRIPTION"].isin(diabetes_conds)]["ENCOUNTER"])


## Potential inclusions

In [ ]:
df = encdata["patient_encounters"]
age_o = set(df[df["AGE@ENC"]>= 45]["ENCOUNTER"]) # old age 😀
df = encdata["observations"]
bmi_h = set(df[(df['DESCRIPTION']=='Body Mass Index') & (df['VALUENUMERIC']>25)]["ENCOUNTER"])
df = encdata["conditions"]
hyperlipidemia = set(df[(df['DESCRIPTION']=='Hyperlipidemia')]["ENCOUNTER"])

In [ ]:
len(bmi_h), len(age_o), len(diabetes), len(hyperlipidemia)

### How do these features/sets relate to each other?

- Use a Venn Diagram to visualize
- __Note__: ellipses are not scaled by the set size

In [ ]:
features = {"diabetes":diabetes, "age":age_o, "bmi":bmi_h, "hyperlipidemia":hyperlipidemia}
venn(features)

### Let's generate a variety of alerts and see how they work

- We will evaluate peformance by the percentage of encounters that generate an alert

#### Our original alert

In [ ]:
a0 = bmi_h.union(age_o)
report_alert_frac(a0, all_enc)

#### Let's generate an alert for everyone over 45 OR (union) with a BMI > 25 that does NOT have a diabetes condition"

In [ ]:
a1 = bmi_h.union(age_o).difference(diabetes)
report_alert_frac(a1, all_enc)

### How about AND (intersection) instead of OR?

In [ ]:
a2 = bmi_h.intersection(age_o).difference(diabetes)
report_alert_frac(a2, all_enc)

## View the alerted cases

In [ ]:
ipw.interact(view_case, data = ipw.fixed(encdata), enc=a2)


## Create your own and explore